In [1]:
import numpy as np
import tensorflow as tf
import keras

Using TensorFlow backend.


## 1. Define Modified VGG-16 Architecture for MNIST

In [2]:
class VGG16:
    def __init__(self, img_size, num_categories, learning_rate):
        self._model = keras.models.Sequential()
        self._add_conv_2d_filter(64, input_shape=img_size, duplicate=2)
        self._add_max_pooling_layer()
        self._add_conv_2d_filter(128, duplicate=2)
        self._add_max_pooling_layer()
        self._add_conv_2d_filter(256, duplicate=3)
        self._add_max_pooling_layer()
#         self._add_conv_2d_filter(512, duplicate=3)
#         self._add_max_pooling_layer()
#         self._add_conv_2d_filter(512, duplicate=3)
#         self._add_max_pooling_layer()
        self._model.add(keras.layers.Flatten())
        self._add_dense_layer(2048, 'relu')
        self._add_dense_layer(2048, 'relu')
        self._add_dense_layer(num_categories, 'softmax')
        self._optimizer = keras.optimizers.Adam(lr=learning_rate)
        self._model.compile(
            loss='categorical_crossentropy',
            optimizer=self._optimizer,
            metrics=['accuracy']
        )
        
    def train(self, x, y, batch_size, epochs, dev_ratio=0.2):
        self._model.fit(
            x=x, y=y, batch_size=batch_size, epochs=epochs, validation_split=dev_ratio, verbose=1
        )
    
    def predict(self, x):
        return np.argmax(self._model.predict(x), axis=-1)
    
    def print_model_summary(self):
        self._model.summary()

    def _add_conv_2d_filter(self, filters, input_shape=None, duplicate=1):
        for idx in range(duplicate):
            kwargs = {
                'filters': filters,
                'kernel_size': 3,
                'strides': 1,
                'padding': 'same',
                'data_format': 'channels_last',
                'activation': 'relu',
                'kernel_initializer': 'glorot_normal'
            }
            if input_shape is not None and idx == 0:
                kwargs['input_shape'] = input_shape
            self._model.add(keras.layers.Conv2D(**kwargs))
    
    def _add_max_pooling_layer(self):
        self._model.add(keras.layers.MaxPool2D(pool_size=2, strides=2))
    
    def _add_dense_layer(self, units, activation):
        self._model.add(keras.layers.Dense(units=units, activation=activation, kernel_initializer='glorot_normal'))

## 2. Get MNIST Dataset

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("data/mnist", one_hot=True, reshape=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/mnist\t10k-images-idx3-ubyte.gz
Extracting data/mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
model = VGG16(
    mnist.train.images.shape[1:],
    mnist.train.labels.shape[-1],
    1e-3
)

In [6]:
model.print_model_summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 256)         295168    
__________

In [7]:
model.train(
    x=mnist.train.images,
    y=mnist.train.labels,
    batch_size=256,
    epochs=50,
    dev_ratio=0.1
)

Train on 49500 samples, validate on 5500 samples
Epoch 1/50
49500/49500 [==============================] - 10s 204us/step - loss: 0.3298 - acc: 0.8941 - val_loss: 0.0430 - val_acc: 0.9878
Epoch 2/50
49500/49500 [==============================] - 7s 146us/step - loss: 0.0534 - acc: 0.9842 - val_loss: 0.0430 - val_acc: 0.9865
Epoch 3/50
49500/49500 [==============================] - 7s 149us/step - loss: 0.0364 - acc: 0.9888 - val_loss: 0.0343 - val_acc: 0.9898
Epoch 4/50
49500/49500 [==============================] - 7s 146us/step - loss: 0.0281 - acc: 0.9913 - val_loss: 0.0364 - val_acc: 0.9898
Epoch 5/50
49500/49500 [==============================] - 7s 146us/step - loss: 0.0244 - acc: 0.9922 - val_loss: 0.0216 - val_acc: 0.9944
Epoch 6/50
49500/49500 [==============================] - 7s 146us/step - loss: 0.0201 - acc: 0.9940 - val_loss: 0.0326 - val_acc: 0.9920
Epoch 7/50
49500/49500 [==============================] - 7s 147us/step - loss: 0.0161 - acc: 0.9951 - val_loss: 0.0205 - 

In [9]:
model.predict(mnist.test.images[:10])

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], dtype=int64)

In [11]:
np.argmax(mnist.test.labels[:10], axis=-1)

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], dtype=int64)